# Класс логистической регресcии

In [110]:
import numpy as np

class LinearRegression:
    def __init__(self):
        self.weights = None
        self.bias = 0

    def fit(self, features, target, learning_rate=0.00001, epochs=30000):
        features = np.array(features)
        target = np.array(target)
        n_samples, n_features = features.shape

        self.weights = np.zeros(n_features)

        for epoch in range(epochs):
            target_predicted = np.dot(features, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(features.T, (target_predicted - target))
            db = (1 / n_samples) * np.sum(target_predicted - target)
            dw = np.clip(dw, -1e4, 1e4)
            db = np.clip(db, -1e4, 1e4)

            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

    def predict(self, features):
        return np.dot(features, self.weights) + self.bias

    @staticmethod
    def calc_metric(target_true, target_pred, metric='rmse'):
        target_true = np.array(target_true)
        target_pred = np.array(target_pred)

        if metric == 'r2':
            ss_total = np.sum((target_true - np.mean(target_true)) ** 2)
            ss_residual = np.sum((target_true - target_pred) ** 2)
            return 1 - (ss_residual / ss_total)
        elif metric == 'rmse':
            return np.sqrt(np.mean((target_true - target_pred) ** 2))

        return None


# Пример обучения модели с использованием реализованного класса

In [111]:
import pandas as pd

data = pd.read_csv(
    "data/cars_prepared.data",
)

print(data.head())


   Unnamed: 0       mpg  cylinders  displacement  horsepower    weight  \
0           0 -0.706439   1.498191      1.090604    0.670894  0.630870   
1           1 -1.090751   1.498191      1.503514    1.588203  0.854333   
2           2 -0.706439   1.498191      1.196232    1.195071  0.550470   
3           3 -0.962647   1.498191      1.061796    1.195071  0.546923   
4           4 -0.834543   1.498191      1.042591    0.932982  0.565841   

   acceleration  model_year    origin  car_name  
0     -1.295498   -1.627426 -0.715145 -1.113745  
1     -1.477038   -1.627426 -0.715145 -1.259185  
2     -1.658577   -1.627426 -0.715145  0.922428  
3     -1.295498   -1.627426 -0.715145 -1.505316  
4     -1.840117   -1.627426 -0.715145  0.139285  


## Разделение выборки на тренировочную и тестовую

In [112]:
target = data['mpg']
features = data.drop(columns=['mpg', 'car_name'])

n_samples = len(features)
test_size = int(n_samples * 0.2)

indices = np.arange(n_samples)
np.random.shuffle(indices)

test_indices = indices[:test_size]
train_indices = indices[test_size:]

features_train = features.iloc[train_indices]
features_test = features.iloc[test_indices]
target_train = target.iloc[train_indices]
target_test = target.iloc[test_indices]

## Обучение модели

In [113]:
model = LinearRegression()

model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print(f"Значение коэффициента детерминации на тренировочной выборке:"
      f" {model.calc_metric(target_train, train_predictions, metric='r2')}")
print(f"Значение коэффициента детерминации на тестовой выборке:"
      f" {model.calc_metric(target_test, test_predictions, metric='r2')}")
print(f"Значение среднеквадратичной ошибки на тренировочной выборке:"
      f" {model.calc_metric(target_train, train_predictions, metric='rmse')}")
print(f"Значение среднеквадратичной ошибки на тестовой выборке:"
      f" {model.calc_metric(target_test, test_predictions, metric='rmse')}")

Значение коэффициента детерминации на тренировочной выборке: 0.7014079486816523
Значение коэффициента детерминации на тестовой выборке: 0.7199890387949139
Значение среднеквадратичной ошибки на тренировочной выборке: 0.5439961703383744
Значение среднеквадратичной ошибки на тестовой выборке: 0.538536806263656
